In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"
# include("../../src/juliaEAM.jl")
include("../../src/lammpsIO.jl")

using Pkg
Pkg.activate(".")

using Printf
using AtomsCalculators
using Unitful: Å, nm
using StaticArrays: SVector
using Molly
using LinearAlgebra
using DelimitedFiles
using UnitfulAtomic
import PeriodicTable
using ProgressMeter
using Random

using LAMMPS

  Activating project at `~/Documents/ABCD_J/EAM/lammps_wrap`


In [2]:
al_LatConst = 4.0495/10 # nm
atom_mass = 26.9815u"u"  # Atomic mass of aluminum in grams per mole

26.9815 u

In [3]:
function initialize_system_dump(;loggers=(coords=CoordinateLogger(1),),filename_dump="")
    n_atoms, box_size, coords_molly, attypes = lmpDumpReader(filename_dump)
    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                    #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                    ) for i in 1:length(coords_molly)]
    # Specify boundary condition
    boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

    atom_positions_init = copy(coords_molly)
    molly_atoms_init = copy(molly_atoms)

    DNF = DistanceNeighborFinder(
        eligible=trues(length(molly_atoms_init), length(molly_atoms_init)),
        n_steps=1e3,
        dist_cutoff=8u"Å")
    TNF = TreeNeighborFinder(
        eligible=trues(length(molly_atoms_init), length(molly_atoms_init)), 
        n_steps=1e3,
        dist_cutoff=8u"Å")

    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms_init,
    atoms_data = [AtomData(element="Al", atom_type=string(attypes[ia])) for (ia,a) in enumerate(molly_atoms_init)],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DNF,
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/Å"  # Ensure these units are correctly specified
    )
    return system_init
end

initialize_system_dump (generic function with 1 method)

In [4]:
filename_dump = "./out_divacancy.dump"
# filename_dump = "./out_vacancy_big.dump"
molly_system = initialize_system_dump(filename_dump = filename_dump)
# neighbors_all = get_neighbors_all(molly_system)

System with 2046 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[32.39519588960027 Å, 32.39519588960027 Å, 32.39519588960027 Å])

In [5]:
for i in 1:length(molly_system.coords)
    random_direction = randn(size(molly_system.coords[i]))
    molly_system.coords[i] += 0.1u"Å" * random_direction*1
end

In [6]:
# Create a LAMMPS instance
lmp = LMP()

# Define the atom style
command(lmp, "units metal")
command(lmp, "dimension 3")
command(lmp, "boundary p p p")
command(lmp, "atom_style atomic")

# Define the simulation box according to molly_system.boundary
boundary = molly_system.boundary
box_x = ustrip(boundary[1])
box_y = ustrip(boundary[2])
box_z = ustrip(boundary[3])
command(lmp, "region box block 0 $(box_x) 0 $(box_y) 0 $(box_z)")
command(lmp, "create_box 1 box")

# Get atom positions, indices and types
ustripped_coords = map(ustrip, molly_system.coords)
pos = convert(Matrix,(reshape(reinterpret(Float64, ustripped_coords),3,:)))
types = map(atoms_data -> parse(Int32, atoms_data.atom_type), molly_system.atoms_data)
indices = map(atoms -> Int32(atoms.index), molly_system.atoms)

# Create atoms in LAMMPS
LAMMPS.create_atoms(lmp, pos, indices, types)

# Set atom type and mass
command(lmp, "mass 1 26.9815")

# Define interatomic potential
command(lmp, "pair_style eam/alloy")
command(lmp, "pair_coeff * * Al99.eam.alloy Al")

# Compute force 
command(lmp, "compute f all property/atom fx fy fz")

command(lmp, "run 0")

f = gather(lmp, "c_f", Float64)
LAMMPS.close!(lmp)

f_transposed = transpose(f)
f_converted = [SVector{3, typeof(1.0u"eV/Å")}(f_transposed[i,:]*1.0u"eV/Å"...) for i in 1:size(f_transposed, 1)]

LAMMPS (29 Aug 2024)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Created orthogonal box = (0 0 0) to (32.395196 32.395196 32.395196)
  1 by 1 by 1 MPI processor grid
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8.28721
  ghost atom cutoff = 8.28721
  binsize = 4.143605, bins = 8 8 8
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d
      bin: standard
Setting up Verlet run ...
  Unit style    : metal
  Current step  : 0
  Time step     : 0.001
Per MPI rank memory allocation (min/avg/max) = 3.935 | 3.935 | 3.935 Mbytes
   Step          Temp          E_pair         E_mol          TotEng         Press     
         0   0             -6727.7196      0             -

2046-element Vector{SVector{3, Quantity{Float64, 𝐋 𝐌 𝐓^-2, Unitful.FreeUnits{(Å^-1, eV), 𝐋 𝐌 𝐓^-2, nothing}}}}:
 [0.6324860069536028 eV Å^-1, 0.23582168964858782 eV Å^-1, 0.32279167480246107 eV Å^-1]
 [-0.553860171422416 eV Å^-1, -1.015962719957699 eV Å^-1, -0.07765470750507238 eV Å^-1]
 [-0.01641272606743365 eV Å^-1, -0.7029391889660167 eV Å^-1, -0.9180115570517074 eV Å^-1]
 [-0.0794594363016417 eV Å^-1, -0.4323191780912867 eV Å^-1, 0.49821447950127973 eV Å^-1]
 [0.04161732717668302 eV Å^-1, 0.022948094175482252 eV Å^-1, 0.1705259535814054 eV Å^-1]
 [-0.5172373153848187 eV Å^-1, -0.17224461990403744 eV Å^-1, 0.7122036515977234 eV Å^-1]
 [0.1661464480390545 eV Å^-1, -0.38500446955868306 eV Å^-1, -0.34538316413588677 eV Å^-1]
 [0.33733994757548214 eV Å^-1, -0.3354124299580391 eV Å^-1, 1.0077403081659877 eV Å^-1]
 [0.12180119816901505 eV Å^-1, 0.6033301645419237 eV Å^-1, -0.48531683788328145 eV Å^-1]
 [-0.6341678290901986 eV Å^-1, 0.4592996170226181 eV Å^-1, -0.22712703716546004 eV Å^-1]

In [7]:
# function calculate_forces(sys::System, pair_coeff_string::String)
#     # Create a LAMMPS instance
#     lmp = LMP(["-screen","none"])

#     # Define the atom style
#     command(lmp, "units metal")
#     command(lmp, "dimension 3")
#     command(lmp, "boundary p p p")
#     command(lmp, "atom_style atomic")

#     # Define the simulation box according to sys.boundary
#     boundary = sys.boundary
#     box_x = ustrip(boundary[1])
#     box_y = ustrip(boundary[2])
#     box_z = ustrip(boundary[3])
#     command(lmp, "region box block 0 $(box_x) 0 $(box_y) 0 $(box_z)")
#     command(lmp, "create_box 1 box")

#     # Get atom positions, indices and types
#     ustripped_coords = map(ustrip, sys.coords)
#     pos = convert(Matrix, (reshape(reinterpret(Float64, ustripped_coords), 3, :)))
#     types = map(atoms_data -> parse(Int32, atoms_data.atom_type), sys.atoms_data)
#     indices = map(atoms -> Int32(atoms.index), sys.atoms)

#     # Create atoms in LAMMPS
#     LAMMPS.create_atoms(lmp, pos, indices, types)

#     # Set atom type and mass
#     command(lmp, "mass 1 26.9815")

#     # Define interatomic potential
#     command(lmp, "pair_style eam/alloy")
#     command(lmp, pair_coeff_string)

#     # Compute force 
#     command(lmp, "compute f all property/atom fx fy fz")

#     command(lmp, "run 0")

#     f = gather(lmp, "c_f", Float64)
#     LAMMPS.close!(lmp)

#     f_transposed = transpose(f)
#     f_converted = [SVector{3, typeof(1.0u"eV/Å")}(f_transposed[i, :] * 1.0u"eV/Å"...) for i in 1:size(f_transposed, 1)]

#     return f_converted
# end

In [8]:
# calculate_forces(molly_system, "pair_coeff * * Al99.eam.alloy Al")

In [9]:
include("./juliaEAM.jl")
## `AtomCalculators` force/energy calculator and `Molly` simulator
eam = EAM()
fname = "Al99.eam.alloy"
read_potential!(eam, fname)

### Define customized interaction type in `AtomsCalculators`
struct EAMInteractionJulia
    calculator::Any  # Holds the ASE EAM calculator reference
    f_energy::Any    # Holds the energy function
    f_forces::Any    # Holds the forces function
    f_atomstress::Any  # Holds the atomic level stresses function
    pair_coeff_string::String  # Holds the pair coefficient string
end

In [10]:
include("./simulator.jl")

eamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces_LAMMPS,calculate_atomstress,"pair_coeff * * Al99.eam.alloy Al")
function Molly.forces(sys::System, interaction::EAMInteractionJulia, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"), neighbors_all::Vector{Vector{Int}};
    n_threads::Integer=Threads.nthreads(), nopenalty_atoms=[]) 
    
    fs = interaction.f_forces(sys, interaction.pair_coeff_string)

    # Add penalty term to forces
    if penalty_coords != nothing
        fs += penalty_forces(sys, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # ev/Å
        # print(maximum(norm.(penalty_forces(sys, penalty_coords, sigma, W))),"\n")
    end
    return fs
end

# eamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces,calculate_atomstress,"pair_coeff * * Al99.eam.alloy Al")
# function Molly.forces(sys::System, interaction::EAMInteractionJulia, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"), neighbors_all::Vector{Vector{Int}};
#     n_threads::Integer=Threads.nthreads(), nopenalty_atoms=[]) 
    
#     fs = interaction.f_forces(interaction.calculator, sys, neighbors_all)

#     # Add penalty term to forces
#     if penalty_coords != nothing
#         fs += penalty_forces(sys, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # ev/Å
#         # print(maximum(norm.(penalty_forces(sys, penalty_coords, sigma, W))),"\n")
#     end
#     return fs
# end

neighbors_all = get_neighbors_all(molly_system)

2046-element Vector{Vector{Int64}}:
 [2, 3, 4, 5, 6, 7, 8, 17, 18, 19  …  1949, 1950, 1951, 2029, 2031, 2041, 2042, 2044, 2045, 2046]
 [1, 3, 4, 5, 6, 7, 8, 9, 12, 17  …  1933, 1934, 1935, 1948, 1949, 1950, 1951, 1966, 2045, 2046]
 [1, 2, 4, 5, 6, 7, 8, 9, 12, 17  …  1279, 1293, 1405, 1600, 1933, 1934, 1935, 1950, 2045, 2046]
 [1, 2, 3, 5, 6, 7, 8, 17, 18, 19  …  1934, 1935, 1946, 1948, 1949, 1950, 1951, 1966, 2045, 2046]
 [1, 2, 3, 4, 6, 7, 8, 9, 10, 11  …  1054, 1149, 1150, 1165, 1277, 1933, 1934, 1950, 2045, 2046]
 [1, 2, 3, 4, 5, 7, 8, 9, 10, 11  …  1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020]
 [1, 2, 3, 4, 5, 6, 8, 9, 10, 11  …  1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020]
 [1, 2, 3, 4, 5, 6, 7, 9, 10, 11  …  1053, 1054, 1149, 1165, 1166, 1181, 1182, 1277, 1934, 1950]
 [2, 3, 5, 6, 7, 8, 10, 11, 12, 13  …  1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
 [5, 6, 7, 8, 9, 11, 12, 13, 14, 15  …  1040, 1043, 1139, 1152, 1168, 1920, 1923, 1936,

In [11]:
# simulator = ABCSimulator(sigma=1.0*u"Å", W=1.0*u"eV", max_steps=1, max_steps_minimize=20, step_size_minimize=5e-3u"ps", tol=1e-6u"eV/Å")
# Minimize_FIRE!(molly_system, simulator, eamJulia, nothing, neighbors_all;
#          n_threads=1, frozen_atoms=[], neig_interval=5, print_nsteps=true,
#          mass=26.9815u"u")

In [12]:
for _ in 1:100
    f_1 = calculate_forces_LAMMPS(molly_system, "pair_coeff * * Al99.eam.alloy Al")
    sleep(0.00001)
end

In [13]:
for _ in 1:100
    f_2 = calculate_forces(eam, molly_system, neighbors_all)
    sleep(0.00001)
end

In [18]:
# Customized neighbor list calculator
coords = molly_system.coords
bounds = molly_system.boundary
r_cutoff = 8u"Å"

# determine cell number according to the cutoff radius
n_cells = [ceil(Int, bounds[i]/r_cutoff) for i in 1:3]
cell_size = bounds ./ n_cells

# Initialize cell lists
cell_list = [Int[] for _ in 1:prod(n_cells)] # list of cells
cell_indices = fill(0, length(coords))

# Assign atoms to cells
for i in 1:length(coords)
    coords_i = coords[i]
    coords_i = coords_i .- floor.(coords_i ./ bounds) .* bounds
    cell = Tuple(ceil(Int, coords_i[j] / cell_size[j]) for j in 1:3)
    cell_index = (cell[1] - 1) * n_cells[2] * n_cells[3] + (cell[2] - 1) * n_cells[3] + cell[3]
    push!(cell_list[cell_index], i) # Add atom index to cell list
    cell_indices[i] = cell_index    # Record cell index for each atom
end

# Initialize neighbor list
neighbor_list = [Int[] for _ in 1:length(coords)]

# Loop over pairs of cells
for i in 1:length(cell_list)
    c1 = cell_list[i]
    for j in i:length(cell_list)
        c2 = cell_list[j]
        for ii in c1
            rx1, ry1, rz1 = coords[ii]
            for jj in c2
                if ii != jj
                    rx, ry, rz = coords[jj]
                    dx = rx1 - rx
                    dy = ry1 - ry
                    dz = rz1 - rz
                    # Apply periodic boundary conditions
                    dx -= round(dx / bounds[1]) * bounds[1]
                    dy -= round(dy / bounds[2]) * bounds[2]
                    dz -= round(dz / bounds[3]) * bounds[3]
                    rsq = dx^2 + dy^2 + dz^2
                    if rsq < r_cutoff^2
                        push!(neighbor_list[ii], jj)
                        push!(neighbor_list[jj], ii)
                    end
                end
            end
        end
    end
end

In [19]:
neighbors_all = get_neighbors_all(molly_system)

2046-element Vector{Vector{Int64}}:
 [2, 3, 4, 5, 6, 7, 8, 17, 18, 19  …  1949, 1950, 1951, 2029, 2031, 2041, 2042, 2044, 2045, 2046]
 [1, 3, 4, 5, 6, 7, 8, 9, 12, 17  …  1933, 1934, 1935, 1948, 1949, 1950, 1951, 1966, 2045, 2046]
 [1, 2, 4, 5, 6, 7, 8, 9, 12, 17  …  1279, 1293, 1405, 1600, 1933, 1934, 1935, 1950, 2045, 2046]
 [1, 2, 3, 5, 6, 7, 8, 17, 18, 19  …  1934, 1935, 1946, 1948, 1949, 1950, 1951, 1966, 2045, 2046]
 [1, 2, 3, 4, 6, 7, 8, 9, 10, 11  …  1054, 1149, 1150, 1165, 1277, 1933, 1934, 1950, 2045, 2046]
 [1, 2, 3, 4, 5, 7, 8, 9, 10, 11  …  1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020]
 [1, 2, 3, 4, 5, 6, 8, 9, 10, 11  …  1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020]
 [1, 2, 3, 4, 5, 6, 7, 9, 10, 11  …  1053, 1054, 1149, 1165, 1166, 1181, 1182, 1277, 1934, 1950]
 [2, 3, 5, 6, 7, 8, 10, 11, 12, 13  …  1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
 [5, 6, 7, 8, 9, 11, 12, 13, 14, 15  …  1040, 1043, 1139, 1152, 1168, 1920, 1923, 1936,